In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/quazar/anaconda3/env/torchrl_up/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
GAMMA = 0.99

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.env.GymEnv("InvertedPendulum-v2")
runner = rw.runner.SingleRunner(env)
batcher = rw.batcher.RolloutBatcher(runner=runner, batch_size=512)

Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.mean = nn.Linear(64, num_outputs)
        self.log_std = nn.Parameter(torch.zeros(1, num_outputs))
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        mean = self.mean(x)
        log_std = self.log_std.expand_as(mean)
        return mean, log_std        

In [6]:
class GaussianPolicy(rw.policy.BasePolicy):
    def create_dist(self, state):
        mean, log_std = self.nn(state)
        return rw.distributions.Normal(loc=mean, scale=log_std.exp())
    
    def get_action(self, state, step):
        dist = self.create_dist(state)
        return U.to_np(dist.sample())

In [7]:
p_nn = PolicyNN(batcher.state_info.shape[0], batcher.action_info.shape[0]).to(device)
policy = GaussianPolicy(p_nn)

p_opt = torch.optim.Adam(p_nn.parameters(), lr=1e-3)

In [8]:
logger = U.Logger('logs/inverted_pendulum/pg-v0-1')
last_logged_step = 0

Writing logs to: logs/inverted_pendulum/pg-v0-0


In [9]:
for batch in batcher.get_batches(max_steps=5e5, act_fn=policy.get_action):
    batch = batch.to_tensor()
    # TODO: Calculation is actually a little wrong here, v_t_last should not be None
    ret = U.estimators.discounted_sum_rewards(rewards=batch.reward, dones=batch.done, gamma=GAMMA)
    batch = batch.concat_batch()
    
    # Calculate loss
    dist = policy.create_dist(batch.state_t)
    log_prob = dist.log_prob(batch.action)
    assert ret.shape == log_prob.shape
    losses = ret * log_prob
    loss = -losses.mean()
    
    # Optimize
    p_opt.zero_grad()
    loss.backward()
    p_opt.step()
    
    # Write logs    
    if batcher.num_steps > last_logged_step:
        last_logged_step = batcher.num_steps + 50000
        batcher.write_logs(logger)
        logger.add_log('policy/loss', loss)
        logger.log(step=batcher.num_steps)    


--------------------------------------------------------------
Env/Reward/Episode (New)                                | 7.08
Env/Length/Episode (New)                                | 7.08
Env/Reward/Episode (Last 50)                            | 7.60
Env/Length/Episode (Last 50)                            | 7.60
policy/loss                                             | 6.50
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                               | 17.44
Env/Length/Episode (New)                               | 17.44
Env/Reward/Episode (Last 50)                           | 41.68
Env/Length/Episode (Last 50)                           | 41.68
policy/loss                                            | 28.74
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            